# DESaster Vers -  0.1

In [1]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
import pandas as pd
import numpy as np
import desaster
from desaster import request, entities, rebuild, config, io


##### instantiate a simpy environment

In [2]:
sim = simpy.Environment()

In [3]:
def simulation(sim, res, entity):
    
    yield sim.process(request.inspection(entity, 
                                       sim, 
                                       res))
    #print ("Inspection Started for {0} at {1}".format(entity.name, sim.now))
    
    yield sim.process(request.file_insurance_claim(entity,
                                         sim,
                                         res))
    #print ("Insurance Started for {0} at {1}".format(entity.name, sim.now))
    
    yield (sim.process(request.fema_assistance(entity,
                                            sim,
                                            res)) &
           
          sim.process(request.loan(entity, 
                                   sim, 
                                   res)) &
                                            #THESE ARE ALL IN THE SAME YIELD STATEMENT
          sim.process(request.permit(entity, 
                                     sim, 
                                     res)))
    #print ("PreBuilding processes Started for {0} at {1}".format(entity.name, sim.now))
    
    
    yield sim.process(rebuild.rebuild_house(entity,
                                sim,
                                res.contractors))
    #print ("Rebuild Started for {0} at {1}".format(entity.name, sim.now))
    
    yield sim.process(request.engineering_assessment(entity,
                                                    sim,
                                                    res))
    #print ("Engineering assessment Started for {0} at {1}".format(entity.name, sim.now))
    

In [4]:
human_cap_data = {"inspectors": 100,
        "insurance adjusters": 10,
        "fema processors": 30,
        "permit processors": 3,
        "contractors": 25,
        "loan processors": 2,
        "engineers": 15,
        "fema aid": 100000000}


In [5]:
res = desaster.Capitals(sim, human_cap_data)

In [6]:
d = io.view_config()

adjuster_mean = 90.0
adjuster_std = 0.0
adjuster_time = 90.0
engineering_assessment_time = 14.0
engineering_mean = 14.0
engineering_std = 0.0
fema_process_mean = 60.0
fema_process_std = 0.0
fema_process_time = 60.0
inspection_mean = 14.0
inspection_std = 0
inspection_time = 14.0
loan_process_mean = 90.0
loan_process_std = 0.0
loan_process_time = 90.0
permit_process_mean = 30.0
permit_process_std = 0.0
permit_process_time = 30.0
sfr_rebuild_mean = 30.0
sfr_rebuild_std = 0.0
sfr_rebuild_time = 30.0


In [7]:
input_path = "../inputs/Final_Maura_Inputs.csv"
households_df = pd.read_csv(input_path)
households_df.head(3)

,Address,Lat,Long,NEAR_FID,NEAR_DIST,FID_1,ID_Number,Tract,Name,YearBuilt,...,None,Slight,Moderate,Extensive,Complete,At_Least_S,At_Least_M,At_Least_E,TotalRep,Damaged
0,1 ULLAKKO RD,46.350970,-123.812630,2993,0.000727,2993,US002994,5.304995e+10,"HILL, DOROTHY M",1975,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,68700.0,1
1,10 KOVEN ROAD,46.367306,-123.816740,203,0.000550,203,US000204,5.304995e+10,"LEINWEBER, MICHAEL D & THERESA",1975,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,119100.0,1
2,10 RYE LANE,46.587673,-123.619364,2934,0.003318,2934,US002935,5.304995e+10,"MARTIN, PATRICIA L",1969,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,49950.0,1


In [8]:
household = {}
for i in households_df.index:
    household[i] = entities.Household(households_df.loc[i])

In [9]:
for i in range(len(household)):
    #print (household[i])
    sim.process(simulation(sim, res, household[i]))

In [10]:
sim.run()

In [15]:
household[0].__getattribute__("insurance_coverage")

0.0

In [12]:
import pprint
length = (len(household))
#for i in range(length):
#    print (household[i].household.Damaged)
count = 0
for j in range(length):
    #print (j)
    for i in dir(household[j]):
        if "get" in i and household[j].assessment_get < 500:
            print (household[j].household.Name)
            print(household[j].__getattribute__(i), " ", i)
            print("-------------------------")
            count += 1
            print (count)

HILL, DOROTHY M
<method-wrapper '__getattribute__' of Household object at 0x106913940>   __getattribute__
-------------------------
1
HILL, DOROTHY M
238.0   assessment_get
-------------------------
2
HILL, DOROTHY M
164.0   assistance_get
-------------------------
3
HILL, DOROTHY M
104.0   claim_get
-------------------------
4
HILL, DOROTHY M
224.0   house_get
-------------------------
5
HILL, DOROTHY M
14.0   inspection_get
-------------------------
6
HILL, DOROTHY M
194.0   loan_time_get
-------------------------
7
HILL, DOROTHY M
134.0   permit_get
-------------------------
8
LEINWEBER, MICHAEL D & THERESA
<method-wrapper '__getattribute__' of Household object at 0x106913b38>   __getattribute__
-------------------------
9
LEINWEBER, MICHAEL D & THERESA
238.0   assessment_get
-------------------------
10
LEINWEBER, MICHAEL D & THERESA
164.0   assistance_get
-------------------------
11
LEINWEBER, MICHAEL D & THERESA
104.0   claim_get
-------------------------
12
LEINWEBER, MICHAEL D

In [13]:
len(res.housing_stock.items)

2000

In [14]:
vars((household[0]))


{'assessment_get': 238.0,
 'assessment_put': 224.0,
 'assistance_get': 164.0,
 'assistance_payout': 0,
 'assistance_put': 104.0,
 'assistance_request': 68700.0,
 'claim_get': 104.0,
 'claim_payout': 0.0,
 'claim_put': 14.0,
 'damage_value': 68700.0,
 'damaged': 1,
 'house_get': 224.0,
 'house_put': 194.0,
 'household': Address          1 ULLAKKO RD
 Lat                    46.351
 Long                 -123.813
 NEAR_FID                 2993
 NEAR_DIST          0.00072734
 FID_1                    2993
 ID_Number            US002994
 Tract               5.305e+10
 Name          HILL, DOROTHY M
 YearBuilt                1975
 Replacemen              45800
 Content_Va              22900
 Latitude              46.3516
 Longitude            -123.812
 Comment           1.00917e+10
 None                    0.004
 Slight                   0.08
 Moderate                0.578
 Extensive               0.327
 Complete                 0.01
 At_Least_S              0.996
 At_Least_M              0.91